In [1]:
import pandas as pd
import numpy as np


In [2]:
df1=pd.read_csv("Reddit_Data.csv")
df2=pd.read_csv("Twitter_Data.csv")

In [3]:
df1

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1
...,...,...
37244,jesus,0
37245,kya bhai pure saal chutiya banaya modi aur jab...,1
37246,downvote karna tha par upvote hogaya,0
37247,haha nice,1


In [4]:
df2

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [5]:
df = pd.concat([df1, df2], axis=0)

# Shuffle
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


In [6]:
df

,clean_comment,category,clean_text
0,NaN,1.0,seems like modi has given ahead will keep hot ...
1,NaN,1.0,those who are asking lakhs from modi kindly wa...
2,NaN,1.0,“chowkidar” tejasvi surya general secretary bj...
3,NaN,0.0,air india boarding pass with modi’ photo kicks...
4,NaN,0.0,congress led upa surgical strike dont air stri...
...,...,...,...
200224,NaN,1.0,the sarabsprldbaspa opportunistic alliance onl...
200225,NaN,0.0,modi not scientist should not take credit but ...
200226,NaN,0.0,modi smriti has degrees than comedian arnold s...
200227,NaN,1.0,all antiindia forces have united stop modi whi...


In [7]:
df = df.drop(columns=["clean_comment"])

In [8]:
df

,category,clean_text
0,1.0,seems like modi has given ahead will keep hot ...
1,1.0,those who are asking lakhs from modi kindly wa...
2,1.0,“chowkidar” tejasvi surya general secretary bj...
3,0.0,air india boarding pass with modi’ photo kicks...
4,0.0,congress led upa surgical strike dont air stri...
...,...,...
200224,1.0,the sarabsprldbaspa opportunistic alliance onl...
200225,0.0,modi not scientist should not take credit but ...
200226,0.0,modi smriti has degrees than comedian arnold s...
200227,1.0,all antiindia forces have united stop modi whi...


In [9]:
df = df.rename(columns={
    "clean_text": "text",
    "category": "label"
})

In [10]:
df

,label,text
0,1.0,seems like modi has given ahead will keep hot ...
1,1.0,those who are asking lakhs from modi kindly wa...
2,1.0,“chowkidar” tejasvi surya general secretary bj...
3,0.0,air india boarding pass with modi’ photo kicks...
4,0.0,congress led upa surgical strike dont air stri...
...,...,...
200224,1.0,the sarabsprldbaspa opportunistic alliance onl...
200225,0.0,modi not scientist should not take credit but ...
200226,0.0,modi smriti has degrees than comedian arnold s...
200227,1.0,all antiindia forces have united stop modi whi...


In [11]:
df.isnull().sum()

label        7
text     37253
dtype: int64

In [12]:
df = df.dropna()

In [13]:
df.shape

(162969, 2)

In [14]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [15]:
df = df.copy()

# keep only valid sentiment labels
df = df[df["label"].isin([-1, 0, 1])]
df["label"] = df["label"].astype(int)

# sanity check
print(df["label"].value_counts())
print(df["label"].unique())


label
 1    72249
 0    55211
-1    35509
Name: count, dtype: int64
[ 1  0 -1]


In [16]:
from sklearn.model_selection import train_test_split

X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    analyzer="char_wb",
    ngram_range=(3, 5),
    max_features=8000,
    min_df=1
)


In [26]:
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [27]:
from sklearn.svm import LinearSVC

model = LinearSVC(
    class_weight="balanced",
    C=1.0,
    random_state=42
)

model.fit(X_train_vec, y_train)


,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,verbose,0
,random_state,42


In [28]:
from sklearn.metrics import classification_report, f1_score, confusion_matrix

y_pred = model.predict(X_test_vec)

print("Macro F1:", f1_score(y_test, y_pred, average="macro"))
print("Weighted F1:", f1_score(y_test, y_pred, average="weighted"))

print("\nClassification Report:")
print(classification_report(
    y_test,
    y_pred,
    labels=[-1, 0, 1],
    target_names=["Negative", "Neutral", "Positive"]
))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred, labels=[-1, 0, 1]))


Macro F1: 0.8682577591621627
Weighted F1: 0.878728903418136

Classification Report:
              precision    recall  f1-score   support

    Negative       0.80      0.81      0.81      7102
     Neutral       0.88      0.92      0.90     11042
    Positive       0.91      0.88      0.90     14450

    accuracy                           0.88     32594
   macro avg       0.87      0.87      0.87     32594
weighted avg       0.88      0.88      0.88     32594


Confusion Matrix:
[[ 5739   566   797]
 [  433 10184   425]
 [  974   758 12718]]


In [36]:
sample = vectorizer.transform(["i am good"])
model.predict(sample)


array([1])

In [38]:
import joblib

joblib.dump(model, "sentiment_model7.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer7.pkl")


['tfidf_vectorizer7.pkl']

['.anaconda',
 '.android',
 '.antigravity',
 '.bash_history',
 '.conda',
 '.continuum',
 '.emulator_console_auth_token',
 '.expo',
 '.gemini',
 '.gitconfig',
 '.gradle',
 '.ipynb_checkpoints',
 '.ipython',
 '.jupyter',
 '.lesshst',
 '.matplotlib',
 '.ollama',
 '.phvm',
 '.streamlit',
 '.virtual_documents',
 '.vscode',
 'anaconda3',
 'AndroidStudioProjects',
 'ansel',
 'Antigravity',
 'AppData',
 'Application Data',
 'archive (3).zip',
 'archive (4).zip',
 'archive (7).zip',
 'archive (9).zip',
 'bank_transaction_analysis.ipynb',
 'bank_transcation_analysis_real.ipynb',
 'bioreactor_experimental_data.xlsx',
 'Contacts',
 'Cookies',
 'credentials.json',
 'CrossDevice',
 'dev_hack',
 'Documents',
 'Downloads',
 'Favorites',
 'final_model.pkl',
 'gmail-ai-assistant',
 'hdfc_upi_transactions.csv',
 'heart.csv',
 'Links',
 'Local Settings',
 'mlflow.db',
 'mlruns',
 'Music',
 'My Documents',
 'NetHood',
 'NTUSER.DAT',
 'ntuser.dat.LOG1',
 'ntuser.dat.LOG2',
 'NTUSER.DAT{6f35e113-bb4a-11f0-9f